## Project Overview



## Import Libraries

In [1]:
import pandas as pd
import geopandas as gpd


# import local modules
import prospectivity_heatmap


ModuleNotFoundError: No module named 'prospectivity_heatmap'